**特征码41 0F BE 72 10找到 地址  大概是这种东西  13EED4A1054 - 41 0FBE 72 10         - movsx esi,byte ptr [r10+10]

In [209]:
import pymem
from pymem.pattern import pattern_scan_all


def aob_injection(process, aob):
    pm = pymem.Pymem(process)  # Open a handle to the process
    aob_address = pattern_scan_all(
        pm.process_handle, aob
    )  # Scan the entire memory of the process for the AOB
    print(aob_address)  # Print the address where the AOB was found
    print(hex(aob_address))

    return aob_address


#aob = bytes.fromhex("41 0F BE 71 10 45")  # Define the AOB r9+10
aob = bytes.fromhex("41 0F BE 72 10 45 0F")  # Define the AOB r10+10
aob_address=aob_injection(
    "javaw.exe", aob
)  # Call the function with the name of the process and the AOB
#打印出来的地址需要变成jmp

1996898335956
0x1d0f06a7cd4


开辟空间 叫他TR

In [210]:
pm = pymem.Pymem("javaw.exe")
TR = pymem.memory.allocate_memory(pm.process_handle, 4)
print(TR)
print(hex(TR))

1995211472896
0x1d08bdf0000


In [211]:
def format_address(address, length=8):
    reversed_bytes = address.to_bytes(length, byteorder='little')
    reversed_formatted = ' '.join(f'{b:02X}' for b in reversed_bytes)
    return reversed_formatted

#print(format_address(0x1CA99861000))
TR_address_reversed_formatted=format_address(TR)
print(TR_address_reversed_formatted)


00 00 DF 8B D0 01 00 00


In [212]:
push_rax=['50']
#lea_eax_rx_x=['41','8D','41','10']# r9+10
lea_eax_rx_x=['41','8D','42','10']# r10+10
move_TR_eax=['A3']+TR_address_reversed_formatted.split(' ')# [TR],EAX
pop_rax=['58']
#movsx_esi_byte_ptr_rx_x=['41','0F' ,'BE', '71', '10']
movsx_esi_byte_ptr_rx_x=['41','0F' ,'BE', '72', '10']


In [213]:
len('50 41 8D 41 10 A3 00 10 86 99 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC OB'.split(' '))
#'50 41 8D 41 10 A3 00 10 86 99 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC OB'.split(' ')

25

In [214]:
part_head_combine= push_rax+lea_eax_rx_x+move_TR_eax+pop_rax+movsx_esi_byte_ptr_rx_x
part_head_combine
print('len part_head_combine',len(part_head_combine))
print(part_head_combine)
jmp_place=['E9']

len part_head_combine 20
['50', '41', '8D', '42', '10', 'A3', '00', '00', 'DF', '8B', 'D0', '01', '00', '00', '58', '41', '0F', 'BE', '72', '10']


In [215]:
shell_code_len=len(part_head_combine)+1+4
print('shell_code_len',shell_code_len)

shell_code_len 25


In [216]:
# Allocate new memory
newmem = pymem.memory.allocate_memory(pm.process_handle, shell_code_len)

In [217]:
print(newmem)
print(hex(newmem))

1995211407360
0x1d08bde0000


In [218]:
newmen_end_row_address_jmp=hex(len(part_head_combine)+newmem)
newmen_end_row_address_jmp

'0x1d08bde0014'

In [219]:
import struct

def calculate_jmp_offset(jmp_addr, target_addr):
    offset = target_addr - (jmp_addr + 5)
    return offset

def to_signed_32_bit_le(i):
    return struct.pack('<i', i)

In [220]:
# 先算 newmen中jmp 的偏移量
jmp_addr_1=int(newmen_end_row_address_jmp,16)  #多数以14结尾
print('jmp_addr_1',jmp_addr_1)
target_addr_1=aob_address+5
print('target_addr_1',target_addr_1)
print(hex(target_addr_1))
print('offset_1',jmp_addr_1, target_addr_1)
offset_1 = calculate_jmp_offset(jmp_addr_1, target_addr_1)
print('offset_1',offset_1)
offset_1_le = to_signed_32_bit_le(offset_1)
print('offset_1_le',offset_1_le)

# offset_2_le 是从 aob_address 跳到 newmen 的偏移量 A开头
jmp_addr_2=aob_address  # aob address
print('jmp_addr_2',jmp_addr_2,hex(jmp_addr_2))
target_addr_2=newmem
print('target_addr_2',target_addr_2,hex(target_addr_2))
offset_2 = calculate_jmp_offset(jmp_addr_2, target_addr_2)
# Convert to signed 32-bit little endian format
offset_2_le = to_signed_32_bit_le(offset_2)
# Print the offset in hexadecimal
print(offset_2_le.hex())


jmp_addr_1 1995211407380
target_addr_1 1996898335961
0x1d0f06a7cd9
offset_1 1995211407380 1996898335961
offset_1 1686928576
offset_1_le b'\xc0|\x8cd'
jmp_addr_2 1996898335956 0x1d0f06a7cd4
target_addr_2 1995211407360 0x1d08bde0000
2783739b


In [221]:
hex_values=part_head_combine+jmp_place+[offset_1_le[0:1],offset_1_le[1:2],offset_1_le[2:3],offset_1_le[3:4]]
hex_values

['50',
 '41',
 '8D',
 '42',
 '10',
 'A3',
 '00',
 '00',
 'DF',
 '8B',
 'D0',
 '01',
 '00',
 '00',
 '58',
 '41',
 '0F',
 'BE',
 '72',
 '10',
 'E9',
 b'\xc0',
 b'|',
 b'\x8c',
 b'd']

In [222]:
#hex_string = "50 41 8D 41 10 A3 00 00 10 88 CA 01 00 00 58 41 0F BE 71 10 E9 40 52 AC 0B"
#hex_values = hex_string.split()  # split the string into a list of two-character strings
hex_values=part_head_combine+jmp_place+[offset_1_le[0:1],offset_1_le[1:2],offset_1_le[2:3],offset_1_le[3:4]]
byte_values = []
for hv in hex_values:
    if isinstance(hv, str):  # if hv is a hex string
        byte_values.append(int(hv, 16))
    elif isinstance(hv, bytes):  # if hv is a byte object
        byte_values.append(int.from_bytes(hv, 'little'))  # or 'big' if your data is big-endian
shellcode = bytes(byte_values)
print(shellcode)


b'PA\x8dB\x10\xa3\x00\x00\xdf\x8b\xd0\x01\x00\x00XA\x0f\xber\x10\xe9\xc0|\x8cd'


先要创造shellcode 然后再写入

In [223]:
hex(newmem)

'0x1d08bde0000'

In [224]:
result1=pm.write_bytes(newmem, shellcode, len(shellcode))

In [225]:
#现在要去aob address jmp 到newmen
inject_hex = ['E9']+[offset_2_le[0:1],offset_2_le[1:2],offset_2_le[2:3],offset_2_le[3:4]]
print(inject_hex)
byte_values = []
for hv in inject_hex:
    if isinstance(hv, str):  # if hv is a hex string
        byte_values.append(int(hv, 16))
    elif isinstance(hv, bytes):  # if hv is a byte object
        byte_values.append(int.from_bytes(hv, 'little'))  # or 'big' if your data is big-endian
inject_hex_shellcode = bytes(byte_values)
print(inject_hex_shellcode)

['E9', b"'", b'\x83', b's', b'\x9b']
b"\xe9'\x83s\x9b"


In [226]:
inject_hex

['E9', b"'", b'\x83', b's', b'\x9b']

In [227]:
print('aob_address',aob_address,hex(aob_address))

aob_address 1996898335956 0x1d0f06a7cd4


In [228]:
len(inject_hex_shellcode)
print(inject_hex_shellcode)

b"\xe9'\x83s\x9b"


In [229]:
inject_hex_shellcode

b"\xe9'\x83s\x9b"

In [230]:
print(aob_address, inject_hex_shellcode, len(inject_hex_shellcode))

1996898335956 b"\xe9'\x83s\x9b" 5


In [231]:
result2=pm.write_bytes(aob_address, inject_hex_shellcode, len(inject_hex_shellcode))
print(result2)

None


下面有点意思

In [74]:
import struct
def offset_to_dec(offset):

  offset_bytes = bytes.fromhex(offset)
  offset = struct.unpack('<i', offset_bytes)[0]
  return offset

offset = offset_to_dec('A7AD53F4')
print('aob 跳 newmen',offset)
print(hex(offset).upper())

offset = offset_to_dec('4052AC0B')
print('newmen结尾跳 aob+5',offset)
print(hex(offset).upper())



aob 跳 newmen -195842649
-0XBAC5259
newmen结尾跳 aob+5 195842624
0XBAC5240


jmp_addr_1 1969670717460
target_addr_1 1969866560089
4052ac0b
jmp_addr_2 1969866560084
target_addr_2 1969670717440
a7ad53f4


In [60]:
offset_hex = 'A7AD53F4'
offset_bytes = bytes.fromhex(offset_hex)
offset = struct.unpack('<i', offset_bytes)[0]

print(offset)
print(hex(offset))

-195842649
-0xbac5259


In [ ]:
1CAA5325254 - bac5254 =1CA 9986 0000

In [61]:
def offset_to_hex(offset):
    packed = struct.pack('<i', offset)  # Pack the offset as a little-endian signed int
    return packed.hex()  # Convert the bytes to a hex string

# Example usage:
offset = 195842644
hex_offset = offset_to_hex(offset)
print(hex_offset)  # Should output '4052ac0b'

5452ac0b


In [48]:
# 'BAC5254'to 32 bit int
BAC5254_32_bit_int=int('BAC5254', 16)
print(hex(BAC5254_32_bit_int))


0xbac5254


In [ ]:
# 1CAA5325254-1CA99860014 = 0xbac5254 or 195,842,629

In [49]:
def calculate_jump_offset(jmp_address, destination):
    destination = int(destination, 16)  # Convert the destination from hexadecimal to an integer

    offset = destination - jmp_address - 5  # Subtract 5 because the relative offset is from the next instruction after the `jmp`, which is 5 bytes later

    # If the offset is negative, convert it to an unsigned integer
    if offset < 0:
        offset += 0x100000000

    return hex(offset)  # Return the offset as a hexadecimal string

# Example usage
jmp_address = 0x1CAA5325254
destination = '1CA99860000'
print(calculate_jump_offset(jmp_address, destination))  # Output: 0xA7AD53F4


0xf453ada7


In [ ]:
aob-newmen = aob's jump 

In [46]:
def calculate_jump_destination(jmp_address, offset):
    offset = int(offset, 16)  # Convert the offset from hexadecimal to an integer

    # Convert to a signed integer if the offset is larger than the maximum positive value for a 32-bit integer
    if offset > 0x7FFFFFFF:
        offset -= 0x100000000

    destination = jmp_address + offset + 5  # Add 5 because the relative offset is from the next instruction after the `jmp`, which is 5 bytes later

    return hex(destination)  # Return the destination address as a hexadecimal string

# Example usage
jmp_address = 0x1CAA5325254
offset = 'A7AD53F4'
print(calculate_jump_destination(jmp_address, offset))  # Output: 0x1ca99860000


0x1ca4cdfa64d


In [53]:
#195842644  to a  32 bit signed int
# Convert hex to unsigned integer
value = int('A7AD53F4', 16)
value

2813154292

In [34]:
current_address = 0x13EE1A40014
relative_offset = 0x4010A60B

# 计算目标地址
target_address = current_address + relative_offset

# 打印结果
print("目标地址:", hex(target_address))


目标地址: 0x13f21b4a61f


In [32]:
newmem

1368048664576

In [33]:
jump_inst = b"\xE9" + (hex(newmem) - (aob + 5)).to_bytes(4, byteorder='little', signed=True)


TypeError: can't concat int to bytes

In [26]:
print(newmem)
print(hex(newmem))

1368048664576
0x13e860e0000


插入 shell code

In [27]:
result1=pm.write_bytes(newmem, shellcode, len(shellcode))

In [ ]:

print(result1)


In [ ]:
result2=pm.write_uint(aob_address, newmem)
print(result2)

In [ ]:
def read_memory(process, address, length):
    pm = pymem.Pymem(process)
    data = pm.read_bytes(address, length)
    return data

# Convert the address from a hexadecimal string to an integer
address = int('0x1811ecc7854', 16)

# Read 2 bytes from the memory address
data = read_memory('javaw.exe', address, 32)

# Print the data
print(data)

b'\xe9\xa7\x87Q\xf4E\x0f\xb6Q\x12D\x8bB$E\x0f\xb6I\x11A\xc1\xe2\x10A\xc1\xe1\x08A\x0b\xf1A\x0b'


In [ ]:
data_int = int.from_bytes(data, byteorder='little')  # or 'big', depending on the data's endianness
print(data_int)

5091950017904467859766155064152094721085811349173482434888371411639559759849


In [ ]:
# data to hex
data_hex = format(data_int, 'X')
data_hex

'B41F10B4108E1C14110E2C1411149B60F4524428B441251B60F45F45187A7E9'

In [ ]:
import pymem
import pymem.pattern

def aob_injection(process, aob, shellcode):
    pm = pymem.Pymem(process)
    aob_address = pymem.pattern.pattern_scan_all(pm.process_handle, pattern=aob)
    if not aob_address:
        return None, None
    
    # Allocate new memory
    newmem = pymem.memory.allocate_memory(pm.process_handle, len(shellcode))  
      
    # Allocate memory for TR
    TR = pymem.memory.allocate_memory(pm.process_handle, 4)

    # Write shellcode to new memory
    pm.write_bytes(newmem, shellcode, len(shellcode))
    
    # Write the address of newmem at the location of the AOB
    pm.write_uint(aob_address, newmem)
    
    return newmem, TR  # so we can deallocate them later

def deallocate(process, newmem, TR):
    pm = pymem.Pymem(process)
    pymem.memory.free_memory(pm.process_handle, newmem)
    pymem.memory.free_memory(pm.process_handle, TR)

# Example usage
aob = bytes.fromhex('41 0F BE 71 10 45')
shellcode = b'\x90' * 100  # replace with your actual shellcode
process = 'javaw.exe'
newmem, TR = aob_injection(process, aob, shellcode)

# ... do something ...

deallocate(process, newmem, TR)


In [ ]:
import pymem
import pymem.pattern
aob = bytes.fromhex('45 0F BE 4B 10 41 0F B6')
process = 'javaw.exe'
pm = pymem.Pymem(process)
aob_address = pymem.pattern.pattern_scan_all(pm.process_handle, pattern=aob)
print(aob_address)
print(hex(aob_address))

2901531490856
0x2a390c0fa28


In [ ]:
# Allocate memory for TR
TR = pymem.memory.allocate_memory(pm.process_handle, 4)

2901925429248
0x2a3a83c0000


b'PA\x8dC\x10\xa3\x00\x00<\xa8\xa3\x02\x00\x00X'
2901942075392


In [ ]:
print(hex(newmem))

0x2a3a93a0000


In [12]:
import struct
pm = pymem.Pymem('javaw.exe')  # replace 'YourProcessName' with the name of your process
# Register r10 value
r10_value = 'EAC9F058'
r10_dec = int(r10_value, 16)

# Try to read 1 byte from the memory location at r10_dec + 0x10
try:
    value = pm.read_bytes(r10_dec + 0x64,32)
    print(value)
    chunks = [value[i:i+2] for i in range(0, len(data), 2)]

    # Use struct.unpack to interpret each chunk as a 16-bit integer
    ints = [struct.unpack('<H', chunk)[0] for chunk in chunks]

    print(ints)
except Exception as e:
    print(f"Could not read memory: {e}")


b'\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00G\t\x04 \xf1\xd5\x98=\x8em4>\xe4\xff\xc3\xbe\xef^g?'
Could not read memory: name 'data' is not defined


None
None


In [19]:
from keystone import *

CODE = b"""push rax
lea eax,[r11+0x10]
movabs ds:0x2a3a83c0000,eax
pop rax"""

try:
    # Initialize Keystone engine
    ks = Ks(KS_ARCH_X86,  KS_MODE_64)

    # Assemble the CODE
    encoding, count = ks.asm(CODE)
    print("%s = %s" %(CODE, encoding))
except KsError as e:
    print("ERROR: %s" %e)


b'push rax\nlea eax,[r11+0x10]\nmovabs ds:0x2a3a83c0000,eax\npop rax' = [80, 65, 141, 67, 16, 163, 0, 0, 60, 168, 163, 2, 0, 0, 88]


In [ ]:
from keystone import *

# separate assembly instructions by ; or \n
CODE = b"""push eax;lea eax, [r11+10];mov [0x83c0000], eax;pop eax;"""
try:
  # Initialize engine in X86-32bit mode
  ks = Ks(KS_ARCH_X86, KS_MODE_32)
  encoding, count = ks.asm(CODE)
  print("%s = %s (number of statements: %u)" %(CODE, encoding, count))
except KsError as e:
  print("ERROR: %s" %e)

ERROR: Cannot find a symbol (KS_ERR_ASM_SYMBOL_MISSING)


In [ ]:
def generate_shellcode(tr_address, register_offset, offset):
    shellcode = bytearray()
    
    # push eax
    shellcode.append(0x50)
    
    # lea eax, [register+offset]
    shellcode.append(0x8D)
    modrm_byte = (0x80 + register_offset) << 3 | 0x04  # ModRM byte with SIB byte
    shellcode.append(modrm_byte & 0xFF)  # Ensure the byte is within range
    shellcode.extend(offset.to_bytes(4, 'little'))  # Offset
    
    # mov [TR], eax
    shellcode.append(0x89)
    shellcode.append(0x88)  # ModRM byte
    shellcode.extend(tr_address.to_bytes(4, 'little'))  # Offset
    
    # pop eax
    shellcode.append(0x58)
    
    return bytes(shellcode)

# 2901531490856
# 0x2a390c0fa28

# Example usage
TR = 0x2a390c0fa28  # Replace with the desired value for TR
r10_offset = 0x10  # Replace with the desired value for r10 offset
r9_offset = 0x10  # Replace with the desired value for r9 offset
r11_offset = 0x10  # Replace with the desired value for r11 offset

shellcode_r10 = generate_shellcode(TR, 0b010, r10_offset)
shellcode_r9 = generate_shellcode(TR, 0b001, r9_offset)
shellcode_r11 = generate_shellcode(TR, 0b011, r11_offset)

import binascii
print(shellcode_r9)
hex_string = binascii.hexlify(shellcode_r9).decode('utf-8')
print(hex_string)
print(shellcode_r11)
hex_string = binascii.hexlify(shellcode_r11).decode('utf-8')
print(shellcode_r11)

OverflowError: int too big to convert

In [ ]:
def generate_shellcode(tr_address, register_offset, offset):
    shellcode = bytearray()
    
    # push eax
    shellcode.append(0x50)
    
    # lea eax, [register+offset]
    shellcode.append(0x8D)
    modrm_byte = (0x80 + register_offset) << 3 | 0x04  # ModRM byte with SIB byte
    shellcode.append(modrm_byte & 0xFF)  # Ensure the byte is within range
    shellcode.extend(offset.to_bytes(4, 'little'))  # Offset
    
    # mov [TR], eax
    shellcode.append(0x89)
    shellcode.append(0x88)  # ModRM byte
    shellcode.extend(tr_address.to_bytes(8, 'little'))  # TR address - adjusted to 8 bytes
    
    # pop eax
    shellcode.append(0x58)
    
    return bytes(shellcode)
# Example usage
TR = 0x2a390c0fa28  # Replace with the desired value for TR
r10_offset = 0x10  # Replace with the desired value for r10 offset
r9_offset = 0x10  # Replace with the desired value for r9 offset
r11_offset = 0x10  # Replace with the desired value for r11 offset

shellcode_r10 = generate_shellcode(TR, 0b010, r10_offset)
shellcode_r9 = generate_shellcode(TR, 0b001, r9_offset)
shellcode_r11 = generate_shellcode(TR, 0b011, r11_offset)

import binascii
# print(shellcode_r9)
# hex_string = binascii.hexlify(shellcode_r9).decode('utf-8')
# print(hex_string)
print(shellcode_r11)
hex_string = binascii.hexlify(shellcode_r11).decode('utf-8')
print(shellcode_r11)

b'P\x8d\x1c\x10\x00\x00\x00\x89\x88(\xfa\xc0\x90\xa3\x02\x00\x00X'
b'P\x8d\x1c\x10\x00\x00\x00\x89\x88(\xfa\xc0\x90\xa3\x02\x00\x00X'


In [ ]:
from capstone import Cs, CS_ARCH_X86, CS_MODE_64

# shellcode = b'P\x8d\x0c\x10\x00\x00\x00\x89\x88xV4\x12X'
# new_code = b"P\x8d\x1c\x10\x00\x00\x00\x89\x88(\xfa\xc0\x90\xa3\x02\x00\x00X"
# 创建Capstone对象
md = Cs(CS_ARCH_X86, CS_MODE_64)
md.detail = True

# 反汇编shellcode
for insn in md.disasm(shellcode, 0x1000):
    print(f"{insn.mnemonic} {insn.op_str}")


push rax
lea eax, [r11 + 0x10]
movabs dword ptr [0x2a3a83c0000], eax
pop rax


In [ ]:
from capstone import Cs, CS_ARCH_X86, CS_MODE_32

# 定义x86汇编代码
shellcode = b"\x50\x8D\x05\x0A\x00\x00\x00\xA3\x00\x00\x3C\xA8\x58"

# 创建Capstone对象
md = Cs(CS_ARCH_X86, CS_MODE_32)

# 反汇编代码
for i in md.disasm(shellcode, 0x1000):
    print("0x%x:\t%s\t%s" % (i.address, i.mnemonic, i.op_str))


0x1000:	push	eax
0x1001:	lea	eax, [0xa]
0x1007:	mov	dword ptr [0xa83c0000], eax
0x100c:	pop	eax


In [ ]:
from capstone import Cs, CS_ARCH_X86, CS_MODE_32

# 创建Capstone对象
md = Cs(CS_ARCH_X86, CS_MODE_32)
TR = 0x12345678
# 汇编指令
assembly_code = f'''
push eax
lea eax, [r9+10]
mov [TR], eax
pop eax
'''

# 汇编代码
instructions = list(md.disasm(assembly_code, 0x1000))

# 获取机器码
machine_code = b''.join([bytes(i.bytes) for i in instructions])

# 打印机器码
print(machine_code)


ArgumentError: argument 2: TypeError: wrong type

In [ ]:
from capstone import Cs, CS_ARCH_X86, CS_MODE_32

# 创建Capstone对象
md = Cs(CS_ARCH_X86, CS_MODE_32)
TR = 0x12345678

# 汇编指令
assembly_code = b'\x50\x8d\x80\x10\x00\x00\x00\x89\x88\x78\x56\x34\x12\x58'

# 汇编代码
instructions = list(md.disasm(assembly_code, 0x1000))

# 获取机器码
machine_code = b''.join([bytes(i.bytes) for i in instructions])

# 打印机器码
print(machine_code)


b'P\x8d\x80\x10\x00\x00\x00\x89\x88xV4\x12X'


In [ ]:
pip install keystone-engine

                                              0.0/1.4 MB ? eta -:--:--
     --------------------                     0.7/1.4 MB 14.8 MB/s eta 0:00:01
     ---------------------------------------- 1.4/1.4 MB 17.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from keystone import Ks, KS_ARCH_X86, KS_MODE_32

# 创建 Keystone 引擎
ks = Ks(KS_ARCH_X86, KS_MODE_32)

# 汇编指令
assembly_code = '''
push eax
lea eax, [r9+10]
mov [0x12345678], eax
pop eax
'''

# 汇编指令转换为字节码
encoding, _ = ks.asm(assembly_code)
machine_code = bytes(encoding)

# 打印字节码
print(machine_code)


KsError: Cannot find a symbol (KS_ERR_ASM_SYMBOL_MISSING)